In [1]:
# !pip install bayesian-optimization

In [2]:
import deepquantum as dq
import deepquantum.photonic.circuit as circuit
import matplotlib.pyplot as plt
import numpy as np

from deepquantum.optimizer import *
from UnitaryDecomp import *
from scipy.stats import unitary_group

np.set_printoptions(precision=8, floatmode='fixed', suppress=True) # to make the print info aligned

N = 8
u8x8 = unitary_group.rvs(N)
decomp_rlt = decomp(u8x8, "cssr")
mzi_info = decomp_rlt[0]

def zero_init(mzi_info):
    for i in range(len(mzi_info['MZI_list'])):
        mzi_info['MZI_list'][i][2] = 0.0
        mzi_info['MZI_list'][i][3] = 0.0
    mzi_info['phase_angle_ori'] = np.zeros(N)
    mzi_info['phase_angle'] = np.zeros(N)

def bar_securing(mzi_info):
    adjustable_ids = []
    for i in range(len(mzi_info['MZI_list'])):
        if i not in adjustable_ids:
            mzi_info['MZI_list'][i][-1] = np.pi
            mzi_info['MZI_list'][i][-2] = np.pi
    
def xx_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2

def yy_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2
        mzi_info['MZI_list'][i][-2] = np.pi

def zz_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi
        mzi_info['MZI_list'][i][-2] = np.pi

def zx_planting(mzi_info):
    mzi_info['MZI_list'][20][-1] = np.pi
    mzi_info['MZI_list'][20][-2] = np.pi
    mzi_info['MZI_list'][24][-1] = np.pi/2


def trial_planting(mzi_info,angle_list):
    change_list = [[1,-1],
                   [4,-1],
                   [5,-1]]
    for i in range(len(angle_list)):
        a,b = change_list[i]
        mzi_info['MZI_list'][a][b] = angle_list[i]
        
zero_init(mzi_info)
bar_securing(mzi_info)
zz_planting(mzi_info)


def post_selection(rlt_strkey):
    # print(rlt_strkey)
    label_list = ['|00101000>', '|00100100>', '|00011000>', '|00010100>']
    value = np.zeros(4)
    for i in range(4):
        try:
            value[i] = rlt_strkey[label_list[i]]
        except:
            pass
    return value


def estimate_energy(post_selection_rlt,type='ZZ'):
    norm = post_selection_rlt/(post_selection_rlt.sum()+1e-16)
    if type=='ZZ': # ZZ basis
        return norm[0]+norm[3]-norm[1]-norm[2]
    if type=='ZI': # ZI basis
        return norm[0]+norm[1]-norm[2]-norm[3]
    if type=='IZ': # IZ basis
        return -norm[0]-norm[1]+norm[2]+norm[3]

def compute_process(mzi_info):
    cir = circuit.QumodeCircuit(nmode=8,init_state=[0,0, 0, 1, 0, 1, 0,0], name="test", cutoff = 3, basis = True) # basis=True, using state list 
    for info in mzi_info['MZI_list']:
        cir.ps(inputs=info[2],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
        cir.ps(inputs=info[3],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
    rlt = cir()
    rlt_strkey = dict()
    for key in rlt.keys():
        rlt_strkey[str(key)] = np.abs(rlt[key].detach().numpy().squeeze())**2

    # rltm = cir.measure(shots=10000)[0]
    # rltm_strkey = dict()
    # for key in rltm.keys():
    #     rltm_strkey[str(key)] = rltm[key]
    # return post_selection(rltm_strkey)

    return post_selection(rlt_strkey)

def estimate_eigval(angle_list):
    # to minimize
    zero_init(mzi_info)
    bar_securing(mzi_info)
    trial_planting(mzi_info,angle_list)

    zz_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_zz = estimate_energy(post_selection_rlt)

    xx_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_xx = estimate_energy(post_selection_rlt)

    yy_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_yy = estimate_energy(post_selection_rlt)
    
    return (1 + e_zz - e_xx - e_yy) / 2  

    # zz_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zi = estimate_energy(post_selection_rlt,type='ZI')

    # xx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_ix = estimate_energy(post_selection_rlt,type='IZ')

    # zx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zx = estimate_energy(post_selection_rlt,type='ZZ')
    
    # return (1 + e_zi + e_ix - e_zx) / 2  


def target_function(angle_list):
    # to maximize
    return -estimate_eigval(angle_list)

In [3]:
# 测试BO算法
print("="*50)
print("运行BO")
param_init = {'t1':0, 't2':0, 't3':0}
bo_optimizer = OptimizerBayesian(target_func=void_target,param_init=param_init,random_state=0)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(300):
    # 待片上计算的参数
    p1 = bo_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    f1 = [-estimate_eigval(p.tolist()) for p in p1]  # BO 内置用法是最大化目标；但是接下来打印时再添一个符号即可
    # 测试结果传回，更新param_dict
    bo_optimizer.param_register(p1,f1)
    f1_to_print = -f1[0]
    if bo_optimizer.best_target == f1[0]:
        mark = "*"
    else:
        mark = " "
    if f1_to_print<0:
        print(bo_optimizer.iter,"\t",p1[0],"\t",f"{f1_to_print:8.10f} ",mark)
    else:
        print(bo_optimizer.iter,"\t",p1[0],"\t ",f"{f1_to_print:8.10f} ",mark)

print("BO结果：",estimate_eigval(list(bo_optimizer.best_param_dict.values())))

运行BO
轮次 	 参数值 	                                 目标值
1 	 [3.44829694 4.49366732 3.78727399] 	  0.3947975023  *
2 	 [0.04130112 0.38026479 0.23805479] 	  0.1257409652  *
3 	 [1.44694751 6.01625953 2.40802370] 	  0.9167351361   
4 	 [3.39985388 3.47527230 4.60283868] 	  0.9266208379   
5 	 [1.22734700 1.73043008 0.54248881] 	 -0.6500056869  *
6 	 [2.09069663 1.41932882 0.00000000] 	 -0.7981057980  *
7 	 [2.88308443 2.97512486 0.00000000] 	 -0.9091536015  *
8 	 [5.18329021 1.69226469 0.00000000] 	  0.8384984143   
9 	 [1.38729617 3.44087474 0.00000000] 	  0.2468443557   
10 	 [2.63693952 2.14578373 0.98234431] 	  0.2032439534   
11 	 [3.36386734 3.65324581 0.00000000] 	 -0.7219093382   
12 	 [1.78492781 0.45702141 0.35235553] 	 -0.3414093964   
13 	 [4.20377718 5.04929688 0.00000000] 	 -0.7100220897   
14 	 [5.48211808 5.61004544 0.00000000] 	 -0.9820747069  *
15 	 [5.03716328 6.28318531 0.99623530] 	  0.5995559393   
16 	 [5.36587593 4.69637828 0.00000000] 	 -0.7862145253   
17 	 [6.23089

In [4]:
# 测试SPSA算法
print("="*50)
print("运行SPSA")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
SPSA_default_hyperparam = {
    'a':1,
    'c':0.1,
    'A':200,
    'nepoch':2000,
    'alpha':0.602,
    'gamma':0.101
}
spsa_optimizer = OptimizerSPSA(target_func=void_target,param_init=param_init,random_state=0,SPSA_hyperparam=SPSA_default_hyperparam)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(200):
    # 待片上计算的两组参数
    p1,p2 = spsa_optimizer.param_suggest()
    # 以下两行模拟芯片计算过程，之后需要替换成接口
    f1 = estimate_eigval(p1.tolist())
    f2 = estimate_eigval(p2.tolist())
    # 测试结果传回，更新param_dict
    spsa_optimizer.param_register([p1,p2],[f1,f2])
    # spsa_optimizer.param_register(np.array([p1,p2]),np.array([f1,f2]))
    

    if f1<0:
        print(spsa_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")
    if f2<0:
        print(spsa_optimizer.iter,"\t",p2,"\t",f"{f2:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p2,"\t ",f"{f2:8.10f} ")
    # print(spsa_optimizer.hyperparam['c']/(1+spsa_optimizer.iter)**spsa_optimizer.hyperparam['gamma'])

    if (f1<-0.92) and (f2<-0.92):
        # print("!!!进入步长减小阶段!!!")
        spsa_optimizer.hyperparam['c'] = 0.001
    elif (f1<-0.999) and (f2<-0.999):
        spsa_optimizer.hyperparam['c'] = 1e-4

print("SPSA结果：",estimate_eigval(list(spsa_optimizer.best_param_dict.values())))

运行SPSA
轮次 	 参数值 	                                 目标值
1 	 [-2.70607696  1.58064184 -0.38688452] 	  0.4638554148 
1 	 [-2.90607696  1.78064184 -0.18688452] 	  0.2980998235 
2 	 [-2.74687241  1.62143729 -0.34608908] 	  0.4347764370 
2 	 [-2.93334970  1.80791458 -0.15961178] 	  0.2701033418 
3 	 [-2.96576409  1.66133403 -0.30619234] 	  0.2197787338 
3 	 [-2.78676915  1.84032897 -0.12719740] 	  0.4515207596 
4 	 [-3.01605186  1.61104626 -0.18261120] 	  0.1394260069 
4 	 [-2.84218295  1.78491517 -0.35648011] 	  0.4048481541 
5 	 [-2.90625268  1.55085127 -0.12241621] 	  0.2323190988 
5 	 [-3.07624685  1.72084544 -0.29241037] 	  0.1055907928 
6 	 [-2.93805621  1.74954726 -0.15421973] 	  0.2487198174 
6 	 [-3.10494867  1.58265480 -0.32111220] 	  0.0652904660 
7 	 [-2.98381707  1.53947220 -0.19998060] 	  0.1620460044 
7 	 [-3.14813127  1.70378640 -0.36429480] 	  0.0393880844 
8 	 [-3.17714731  1.57068941 -0.23119781] 	 -0.0216995892 
8 	 [-3.01503429  1.73280244 -0.39331084] 	  0.2039351334 
9 

In [9]:
# 测试Fourier级数方法
print("="*50)
print("运行Fourier级数法")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
# param_init = dict(zip(param_init.keys(),np.random.randn(3)))
fourier_optimizer = OptimizerFourier(target_func=void_target,param_init=param_init,random_state=0,R=3,lr=0.1)
print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(100):
    # 待片上计算的两组参数
    param_array = fourier_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    target = np.zeros(len(param_array))
    for i in range(len(param_array)):
        target[i] = estimate_eigval(param_array[i])

    # 测试结果传回，更新param_dict
    fourier_optimizer.param_register(param_array,target)
    p1 = np.array(list(fourier_optimizer.best_param_dict.values()))
    f1 = fourier_optimizer.best_target
    if f1<0:
        print(fourier_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(fourier_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")

print("Fourier结果：",estimate_eigval(list(fourier_optimizer.best_param_dict.values())))

运行Fourier级数法
轮次 	 参数值 	                                 目标值
1 	 [ 2.69279370  0.41605005 -1.15618243] 	 -0.0803705552 
2 	 [ 2.69279370  0.43729215 -1.14929049] 	 -0.0848643671 
3 	 [ 2.69279370  0.46207007 -1.14353204] 	 -0.0894169517 
4 	 [ 2.69279370  0.49085238 -1.13875402] 	 -0.0938957727 
5 	 [ 2.69279370  0.52409826 -1.13462876] 	 -0.0981047806 
6 	 [ 2.69279370  0.56221888 -1.13062884] 	 -0.1017938101 
7 	 [ 2.69279370  0.60552903 -1.12603155] 	 -0.1047079836 
8 	 [ 2.69279370  0.65419137 -1.11996259] 	 -0.1066763714 
9 	 [ 1.79519580  0.70815615 -1.11147937] 	 -0.1302699533 
10 	 [ 1.79519580  0.76710035 -1.09967799] 	 -0.1709643070 
11 	 [ 1.79519580  0.83037083 -1.08379986] 	 -0.2132124291 
12 	 [ 1.79519580  0.89694149 -1.06331385] 	 -0.2565053655 
13 	 [ 1.79519580  0.96539965 -1.03795499] 	 -0.3004219837 
14 	 [ 1.79519580  1.03397966 -1.00770951] 	 -0.3447315120 
15 	 [ 1.79519580  1.10065483 -0.97274666] 	 -0.3894329542 
16 	 [ 1.79519580  1.16328160 -0.93331461] 	 -0.4